# __`Support Vector Regression`__ 

In [49]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 

In [50]:
# load Dataset
df = sns.load_dataset('tips')
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [52]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [53]:
for col in df.columns:
    if df[col].dtype == 'category':
        print(f' {col} \n {df[col].unique()}')
        print("---------------------------------")

 sex 
 ['Female', 'Male']
Categories (2, object): ['Male', 'Female']
---------------------------------
 smoker 
 ['No', 'Yes']
Categories (2, object): ['Yes', 'No']
---------------------------------
 day 
 ['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']
---------------------------------
 time 
 ['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']
---------------------------------


In [54]:
# Independent and dependent Features
X = df.drop('total_bill', axis = 1)
y = df['total_bill']

X

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,5.92,Male,No,Sat,Dinner,3
240,2.00,Female,Yes,Sat,Dinner,2
241,2.00,Male,Yes,Sat,Dinner,2
242,1.75,Male,No,Sat,Dinner,2


In [55]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 10)

X_train.shape, X_test.shape

((183, 6), (61, 6))

In [56]:
## Feature Encoding -> (OneHot, Label)

# for col which have only binary classes then we apply label if more than 2 classes then apply OneHotEncoder

from sklearn.preprocessing import LabelEncoder

le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

X_train['sex'] = le1.fit_transform(X_train['sex'])
X_train['smoker'] = le2.fit_transform(X_train['smoker'])
X_train['time'] = le3.fit_transform(X_train['time'])

X_test['sex'] = le1.transform(X_test['sex'])
X_test['smoker'] = le2.transform(X_test['smoker'])
X_test['time'] = le3.transform(X_test['time'])

print(X_train.info())
X_train


<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 58 to 9
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   tip     183 non-null    float64 
 1   sex     183 non-null    int32   
 2   smoker  183 non-null    int32   
 3   day     183 non-null    category
 4   time    183 non-null    int32   
 5   size    183 non-null    int64   
dtypes: category(1), float64(1), int32(3), int64(1)
memory usage: 6.8 KB
None


,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2
...,...,...,...,...,...,...
64,2.64,1,0,Sat,0,3
15,3.92,1,0,Sun,0,2
228,2.72,1,0,Sat,0,2
125,4.20,0,0,Thur,1,6


In [57]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(drop='first'), [3])], 
    remainder='passthrough'
)

X_train = ct.fit_transform(X_train)
X_test  = ct.transform(X_test)

# Convert back to DataFrame with feature names
feature_names = ct.get_feature_names_out()
X_train = pd.DataFrame(X_train, columns=feature_names)
X_test  = pd.DataFrame(X_test,  columns=feature_names)

X_train

,onehot__day_Sat,onehot__day_Sun,onehot__day_Thur,remainder__tip,remainder__sex,remainder__smoker,remainder__time,remainder__size
0,1.0,0.0,0.0,1.76,1.0,1.0,0.0,2.0
1,0.0,1.0,0.0,1.66,1.0,0.0,0.0,3.0
2,0.0,1.0,0.0,3.50,1.0,0.0,0.0,3.0
3,1.0,0.0,0.0,2.01,1.0,0.0,0.0,2.0
4,0.0,1.0,0.0,3.00,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...,...
178,1.0,0.0,0.0,2.64,1.0,0.0,0.0,3.0
179,0.0,1.0,0.0,3.92,1.0,0.0,0.0,2.0
180,1.0,0.0,0.0,2.72,1.0,0.0,0.0,2.0
181,0.0,0.0,1.0,4.20,0.0,0.0,1.0,6.0


In [58]:
## SVR Model Train
from sklearn.svm import SVR
svr = SVR()

svr.fit(X_train, y_train)

SVR()

In [59]:
y_pred = svr.predict(X_test)
y_pred

array([16.63987262, 17.80422691, 14.49145093, 24.62866424, 14.67173361,
       22.67410166, 19.04664805, 28.29108519, 23.75972181, 21.46546953,
       24.86896212, 17.67206073, 13.44339231, 22.32034304, 19.75694037,
       14.95399101, 13.8566174 , 13.39304164, 21.2171004 , 17.63874958,
       14.32207633, 15.49171819, 14.19580813, 13.3369827 , 29.65235934,
       13.8566174 , 17.37720431, 21.38260597, 14.76962827, 30.02183062,
       28.64203024, 28.03548476, 14.79770992, 15.92806049, 17.56421584,
       13.35938349, 20.52423608, 14.51606288, 20.71506125, 22.18900247,
       16.70814899, 21.52705566, 13.2124233 , 24.44552357, 14.05257741,
       14.13643952, 18.52010744, 14.79770992, 13.74271464, 18.54833482,
       20.82086506, 29.78470039, 19.41806679, 27.8646883 , 13.01786739,
       16.91813134, 14.06890087, 14.15277698, 17.87180292, 13.83656792,
       17.37317545])

In [60]:
from sklearn.metrics import r2_score, mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print("Score is :", r2_score(y_test, y_pred))

print("MAE is :", mae)

Score is : 0.4602811456115927
MAE is : 4.1486423210190235


## __`HyperParameter Tuning using GridSearch CV`__

In [61]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100, 1000], 
    'gamma': [1000, 100, 10, 1, 0.1, 0.01], 
    'kernel': ['linear', 'sigmoid', 'rbf']
}

grid = GridSearchCV(estimator = SVR(), param_grid = param_grid, refit = True, verbose = 3)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5] END ..C=0.1, gamma=1000, kernel=linear;, score=0.215 total time=   0.0s
[CV 2/5] END ..C=0.1, gamma=1000, kernel=linear;, score=0.478 total time=   0.0s
[CV 3/5] END ..C=0.1, gamma=1000, kernel=linear;, score=0.717 total time=   0.0s
[CV 4/5] END ..C=0.1, gamma=1000, kernel=linear;, score=0.442 total time=   0.0s
[CV 5/5] END ..C=0.1, gamma=1000, kernel=linear;, score=0.585 total time=   0.0s
[CV 1/5] END C=0.1, gamma=1000, kernel=sigmoid;, score=-0.083 total time=   0.0s
[CV 2/5] END C=0.1, gamma=1000, kernel=sigmoid;, score=-0.074 total time=   0.0s
[CV 3/5] END C=0.1, gamma=1000, kernel=sigmoid;, score=-0.174 total time=   0.0s
[CV 4/5] END C=0.1, gamma=1000, kernel=sigmoid;, score=-0.013 total time=   0.0s
[CV 5/5] END C=0.1, gamma=1000, kernel=sigmoid;, score=-0.111 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=1000, kernel=rbf;, score=-0.088 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=1000, kernel=rbf;

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1000, 100, 10, 1, 0.1, 0.01],
                         'kernel': ['linear', 'sigmoid', 'rbf']},
             verbose=3)

In [62]:
grid.best_params_

{'C': 100, 'gamma': 1000, 'kernel': 'linear'}

In [63]:
y_pred = grid.predict(X_test)

In [64]:
mae = mean_absolute_error(y_test, y_pred)
print("Score is :", r2_score(y_test, y_pred))

print("MAE is :", mae)

Score is : 0.5453723337106238
MAE is : 3.912971558542979
